In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
# import gensim
import joblib
from keras.models import load_model
import gzip
from nltk.tokenize import word_tokenize


In [ ]:
input_file='/Users/M NILESH/Downloads/Label10000.csv'
df=pd.read_csv(input_file)
df=df[['text']]
df['word_count']=df['text'].apply(lambda x: len(str(x).split(" ")))
df.word_count.describe()
common_words=pd.Series(''.join(df['text']).split()).value_counts()
most_common=common_words[:3]
least_common=common_words[-1000:]


In [ ]:
texts=[]
labels=[]
def removeWords(words):
    final_words=[]
    for word in words:
        if not (word in most_common or word in least_common):
            final_words.append(word)
    return final_words
for i in range(len(df)):
    if i%10000==0:
        print("------>",i)
    words=word_tokenize(df.loc[i].text)
    # print(df.loc[i].text)
    words=removeWords(words)
    extracted_sentence=' '.join(word for word in words)
    # print(df.loc[i].text)
    texts.append(extracted_sentence)
new_df=pd.DataFrame({
    'text':texts
})
def tokenize(x):
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk
def pad(x, length=None):
    if length is None:
        length = len(max(x, key=len))

    return pad_sequences(x, maxlen=length, padding='post')


In [ ]:
tokenize_text=[]
news_label=[]
for n_z in new_df['text']:
  tokenize_text.append(n_z)
text_tokenized, text_tokenizer=tokenize(tokenize_text)
text_padded=pad(text_tokenized)
trainX=text_padded
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
refined_out = trainX.astype(np.float)
model = load_model('Bidirectional.h5')
output=model.predict(refined_out)


In [ ]:
newout=[]
for i in range(len(output)):
    if output[i][1]>0.5:
        newout.append(1)
    else:
        newout.append(0)
count1=0
for i in range(len(newout)):
    if newout[i]==y_test[i][1]:
        count1+=1
if (count1>len(newout)-count1):
    print('1')
else:
    print('0')
